## SANS2D reduction
This is a notebook to reduce data from SANS2D. 

### Basic components 
* File loading: sample, sample trans, background, background trans, directbeam, directbeam trans (ess.loki)
* Transform coordinates: sample, bench, monitor position offsets (ess.loki.larmor, consider ess.loki.sans2d, dedicated loader,  helper functions)
* Masking (for file or programatically, ess.loki, masks applied as a part of loader da = ess.loki.load(file) or manually, da.masks['beam_stop'] = ess.loki.sans2d.make_beamstop_mask(da) or ess.loki.mask_beamstop(da)). Paths (not files) to mask to ess.loki.sans2d 
* Instrument geometry offset ess.loki.sans2d.load(.., offset = {position:monitor})
* Define wavelenght and q bins (wavelenght bands) geomspace is fine for the moment
* Reduction (regular full wavelenght range) - ess.sans
* Reduction by wavelenght (reduction by wavelenght through the same fucntion, ess.sans provided loki/sans2d specific functions are removed). groupby argument is LoKI specific. To be looked at
* Normalize and subtract (ess.sans) make more sense to be part of top-level function, both should be fine


### The notebook uses sans.py module that contains

* project_xy don't include 
* solid_angle (ess.loki.larmor and ess.loki.sans2d) -> candidate for scippneutron
* subtract_background_mean
* transmission_fraction - generic function taking monitors as arguments in ess.sans to be called from ess.loki.sans2d. Preprocessing of monitors can be excluded from the main function (independent helper)
* to_wavelength - rebinning may not be required, norm term may be provided as histogram. Lines 87-92 can be moved outside and the call monitor
* reduce uses groupby layer at the moment but may be different argument. Has some specific features of instrument. Spectrum may not be the only choice. ess.loki.sans2d. Eventually can be moved to ess.sans
* reduce_by_wavelength
* q resolution small clean-up (global sample variable)
* gravity - look at the issue

### sans.py calls another module contrib.py

* midpoints 
* to_bin_centers candidate for scipp, copy for now
* to_bin_edges candidate for scipp, copy for now
* map_to_bins - may be better way to implement it, copy for now
* select_bins (not used?)
* make_slices may be nicer way to implement it now


In [1]:
import scipp as sc
import scippneutron as scn
from scipp.plotting import plot
from load_files import load_isis, load_rkh_q, load_rkh_wav, setup_masks
from transform_coordinates import setup_offsets, setup_geometry
import sans

In [2]:
sc.__version__

'0.7.1'

In [3]:
scn.__version__

'0.2.2'

## Loading files

In [4]:
try:
    import dataconfig # run make_config.py to create this
except:
    print("ERROR: dataconfig.py not find please run `make_config.py`\n")
    !python make_config.py -h # change to `-f path` or run in terminal

path = dataconfig.data_root
direct_beam_file = 'DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'
#TODO: Are we using moderator file?
moderator_file = 'ModeratorStdDev_TS2_SANS_LETexptl_07Aug2015.txt'

sample_run_number = 63114
sample_transmission_run_number = 63114
background_run_number = 63159
background_transmission_run_number = 63159 
directbeam_run_number = 63091
directbeam_transmission_run_number = 63091

mask_1 = f'{path}/MASK_SANS2D_REAR_Edges_16Mar2015.xml'
mask_2 = f'{path}/MASK_SANS2D_FRONT_Edges_16Mar2015.xml'
mask_3 = f'{path}/MASK_SANS2D_BOTH_Extras_24Mar2015.xml'
mask_4 = f'{path}/MASK_SANS2D_REAR_Bottom_3_tubes_16May2014.xml'
mask_5 = f'{path}/MASK_SANS2D_beam_stop_4m_x_100mm_2July2015_medium_beamstop.xml'
mask_6 = f'{path}/MASK_SANS2D_REAR_module2_tube12.xml'
mask_7 = f'{path}/MASK_SANS2D_FRONT_module3_tube14_module5_tube6_tube7.xml'
mask_8 = f'{path}/MASK_SANS2D_FRONT_module2_tube18.xml'
mask_9 = f'{path}/MASK_SANS2D_FRONT_module5_tube20.xml'

idf_filename = f'{path}/SANS2D_Definition_Tubes.xml'

#TODO: where are these used?
l_collimation = sc.Variable(value=4.0, unit=sc.units.m)
r2 = sc.Variable(value=4.0/1000, unit=sc.units.m) # sample aperture radius
r1 = sc.Variable(value=10.0/1000, unit=sc.units.m) # source aperture radius (in user file its diameter)  
dr = sc.Variable(value=8.0/1000, unit=sc.units.m) # virtual ring width on detector

In [5]:
%%time
direct_beam = load_rkh_wav(filename=f'{path}/{direct_beam_file}')

#TODO: SANS2D data needs to be trimmed (to be confirmed what is required range spectrum_size =  245760//2 or spectrum_size = 122880//2)
spectrum_size = 122880//2
#spectrum_size = 245760//2
sample = load_isis(filename=f'{path}/SANS2D000{sample_run_number}.nxs', spectrum_size = spectrum_size)
background = load_isis(filename=f'{path}/SANS2D000{background_run_number}.nxs', spectrum_size = spectrum_size)
direct_beam_trans = load_isis(filename=f'{path}/SANS2D000{directbeam_run_number}.nxs', spectrum_size = spectrum_size)
sample_trans = load_isis(filename=f'{path}/SANS2D000{sample_transmission_run_number}.nxs', spectrum_size = spectrum_size)
background_trans = load_isis(filename=f'{path}/SANS2D000{background_transmission_run_number}.nxs', spectrum_size =  spectrum_size)
#TODO: What about units?

FrameworkManager-[Notice] Welcome to Mantid 6.0.20210412.848
FrameworkManager-[Notice] Please cite: http://dx.doi.org/10.1016/j.nima.2014.07.029 and this release: http://dx.doi.org/10.5286/Software/Mantid
CheckMantidVersion-[Notice] A new version of Mantid(6.1.0) is available for download from https://download.mantidproject.org
DownloadInstrument-[Notice] All instrument definitions up to date
Python-[Warning] Failed to load plugin /Users/wojciechpotrzebowski/opt/anaconda3/envs/scippneutron/plugins/python/algorithms/WorkflowAlgorithms/SANS/SANSBeamCentreFinder.py.
Error: Traceback (most recent call last):
  File "/Users/wojciechpotrzebowski/opt/anaconda3/envs/scippneutron/lib/python3.8/site-packages/mantid/kernel/plugins.py", line 201, in load_from_file
    name, module = load_plugin(filepath)
  File "/Users/wojciechpotrzebowski/opt/anaconda3/envs/scippneutron/lib/python3.8/site-packages/mantid/kernel/plugins.py", line 221, in load_plugin
    module = loader.run()
  File "/Users/wojciec

LoadRKH-[Notice] LoadRKH started
LoadRKH-[Notice] LoadRKH successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
Load-[Notice] Load started
Load-[Notice] Load successful, Duration 5.78 seconds


Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'period_change_log' has unrecognised units: 'period_number'
Workspace run log 'raw_frames' has unrecognised units: 'frames'
Workspace run log 'veto_log' has unrecognised units: 'is_vetoing'
Workspace run log 'events_log' has unrecognised units: 'events'
Workspace run log 'frame_log' has unrecognised units: 'frame_number'
Workspace run log 'good_frame_log' has unrecognised units: 'is_good'
Workspace run log 'period_log' has unrecognised units: 'period_number'
Workspace run log 'raw_events_log' has unrecognised units: 'events'


ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] 

Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'period_change_log' has unrecognised units: 'period_number'
Workspace run log 'raw_frames' has unrecognised units: 'frames'
Workspace run log 'veto_log' has unrecognised units: 'is_vetoing'
Workspace run log 'events_log' has unrecognised units: 'events'
Workspace run log 'frame_log' has unrecognised units: 'frame_number'
Workspace run log 'good_frame_log' has unrecognised units: 'is_good'
Workspace run log 'period_log' has unrecognised units: 'period_number'
Workspace run log 'raw_events_log' has unrecognised units: 'events'


...

ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] 

Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'period_change_log' has unrecognised units: 'period_number'
Workspace run log 'raw_frames' has unrecognised units: 'frames'
Workspace run log 'veto_log' has unrecognised units: 'is_vetoing'
Workspace run log 'events_log' has unrecognised units: 'events'
Workspace run log 'frame_log' has unrecognised units: 'frame_number'
Workspace run log 'good_frame_log' has unrecognised units: 'is_good'
Workspace run log 'period_log' has unrecognised units: 'period_number'
Workspace run log 'raw_events_log' has unrecognised units: 'events'


ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] 

Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'period_change_log' has unrecognised units: 'period_number'
Workspace run log 'raw_frames' has unrecognised units: 'frames'
Workspace run log 'veto_log' has unrecognised units: 'is_vetoing'
Workspace run log 'events_log' has unrecognised units: 'events'
Workspace run log 'frame_log' has unrecognised units: 'frame_number'
Workspace run log 'good_frame_log' has unrecognised units: 'is_good'
Workspace run log 'period_log' has unrecognised units: 'period_number'
Workspace run log 'raw_events_log' has unrecognised units: 'events'


ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] ExtractSpectra started
ExtractSpectra-[Notice] ExtractSpectra successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
ExtractSpectra-[Notice] 

In [6]:
sample

<IPython.core.display.Javascript object>

<scipp.DataArray>
Dimensions: Sizes[spectrum:61440, tof:100, ]
Coordinates:
  position                vector_3_float64              [m]  (spectrum)  [(-0.519200, -0.481950, 23.281000), (-0.517162, -0.481950, 23.281000), ..., (0.517162, 0.481950, 23.281000), (0.519200, 0.481950, 23.281000)]
  sample_position         vector_3_float64              [m]  ()  [(0.000000, 0.000000, 19.281000)]
  source_position         vector_3_float64              [m]  ()  [(0.000000, 0.000000, 0.000000)]
  spectrum                    int32  [dimensionless]  (spectrum)  [9, 10, ..., 61447, 61448]
  tof                       float64            [µs]  (tof [bin-edge])  [0.000000, 1000.000000, ..., 99000.000000, 100000.000000]
Data:
                            float32         [counts]  (spectrum, tof)  [0.000000, 0.000000, ..., 0.000000, 0.000000]  [0.000000, 0.000000, ..., 0.000000, 0.000000]
Attributes:
  Changer                 DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:2, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000]
Data:
                             string  [dimensionless]  (time)  ["IT", "JT"]

]
  Det_Temp_FLB            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [16.799999, 16.799999, 16.799999]

]
  Det_Temp_FLT            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.400000, 17.400000, 17.400000]

]
  Det_Temp_FRB            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.400000, 17.400000, 17.400000]

]
  Det_Temp_FRT            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.400000, 17.400000, 17.400000]

]
  Det_Temp_RLB            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.799999, 17.799999, 17.799999]

]
  Det_Temp_RLT            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.400000, 17.400000, 17.400000]

]
  Det_Temp_RRB            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.299999, 17.299999, 17.299999]

]
  Det_Temp_RRT            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  

## Setting up masks and geometries

In [7]:
#TODO: resolve this. Wasn't able to transform coordinates without doing this 
sample = sample['spectrum', 0:spectrum_size].copy()
background = background['spectrum', 0:spectrum_size].copy()
sample_trans = sample_trans['spectrum', 0:spectrum_size].copy()
background_trans = background_trans['spectrum', 0:spectrum_size].copy()
direct_beam_trans = direct_beam_trans['spectrum', 0:spectrum_size].copy()

In [8]:
mask_files = [mask_1, mask_2, mask_3, mask_4, mask_5, mask_6, mask_7, mask_8, mask_9]
setup_masks(idf_filename, mask_files, sample, background, spectrum_size)                                                           
    
#PRINT masking off beamstop arm 15mm wide at 20 degrees
#!PRINT not masking beam stop arm, M4 out
#mask/line 15 20

LoadMask-[Notice] LoadMask started
LoadMask-[Notice] LoadMask successful, Duration 0.29 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
LoadMask-[Notice] LoadMask started
LoadMask-[Notice] LoadMask successful, Duration 0.29 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
LoadMask-[Notice] LoadMask started
LoadMask-[Notice] LoadMask successful, Duration 0.27 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
LoadMask-[Notice] LoadMask started
LoadMask-[Notice] LoadMask successful, Duration 0.27 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
LoadMask-[Notice] LoadMask started
LoadMask-[Notice] LoadMask successful, Duration 0.27 seconds
DeleteWorkspace-[Notice]

### Do coordinate transformations

In [9]:
setup_offsets(sample, sample_trans, background, background_trans, direct_beam_trans)

In [10]:
x = -0.09257
y = 0.081798
z = 0.0
setup_geometry(sample, background, x, y, z)



sample_q_reduce = sans.to_q(data=sample,
                        transmission=sample_trans,
                        direct_beam=direct_beam,
                        direct_beam_transmission=direct_beam_trans, # note: background_trans
                        masks=sample.masks,
                        q_bins = q_bins,
                        wavelength_bins = wavelength_bins)

background_q_reduce = sans.to_q(data=background,
                            transmission=background_trans,
                            direct_beam=direct_beam,
                            direct_beam_transmission=direct_beam_trans, # note: background_trans
                            masks=background.masks,
                            q_bins = q_bins,
                            wavelength_bins = wavelength_bins)

reduced = sans.normalize_and_subtract(sample_q_reduce, background_q_reduce)

sample.coords['position'] = sample.coords['base_position']
background.coords['position'] = background.coords['base_position']

mantid_q1d_file = '63114_rear_1D_1.75_16.5.txt'
mantid_q1d = load_rkh_q(filename=f'{path}/{mantid_q1d_file}')
reduced_scaled = sc.scalar(0.68) * reduced
plot({'scipp': reduced_scaled, 'mantid':mantid_q1d})

## Defining parameters for data reduction 

In [11]:
q_bins = sc.linspace(dim = 'Q', start = 0.01, stop = 0.525, num = 104, unit = sc.units.one/sc.units.angstrom)
wavelength_bins = sc.linspace(dim = 'wavelength', start = 2, stop = 16.5, num = 30, unit=sc.units.angstrom)
wavelength_bands = sc.linspace(dim = 'wavelength', start = 2, stop = 10, num = 9, unit=sc.units.angstrom)
#TODO: Add bands as a predefined list

## Reduction (full spectra)

In [12]:
%%time
sample_q_reduce = sans.to_q(data=sample,
                        background = background, 
                        transmission=sample_trans,
                        direct_beam=direct_beam,
                        direct_beam_transmission=direct_beam_trans, # note: background_trans
                        masks=sample.masks,
                        q_bins = q_bins,
                        wavelength_bins = wavelength_bins)

CPU times: user 308 ms, sys: 228 ms, total: 535 ms
Wall time: 361 ms


In [13]:
%%time
background_q_reduce = sans.to_q(data=background,
                            background = background, 
                            transmission=background_trans,
                            direct_beam=direct_beam,
                            direct_beam_transmission=direct_beam_trans, # note: background_trans
                            masks=background.masks,
                            q_bins = q_bins,
                            wavelength_bins = wavelength_bins)

CPU times: user 291 ms, sys: 223 ms, total: 514 ms
Wall time: 239 ms


In [14]:
reduced = sans.normalize_and_subtract(sample_q_reduce, background_q_reduce)

In [15]:
reduced

<scipp.DataArray>
Dimensions: Sizes[Q:103, ]
Coordinates:
  Q                         float64           [1/Å]  (Q [bin-edge])  [0.010000, 0.015000, ..., 0.520000, 0.525000]
Data:
                            float64  [dimensionless]  (Q)  [2.348278, 3.539430, ..., 0.391624, 0.338739]  [0.000298, 0.000317, ..., 0.001042, 0.001122]
Attributes:
  sample_position         vector_3_float64              [m]  ()  [(0.000000, 0.000000, 19.334000)]
  source_position         vector_3_float64              [m]  ()  [(0.000000, 0.000000, 0.000000)]

## Reduction by wavelength

In [16]:
%%time
sample_q_lambda = sans.to_q(data=sample,
                            background = background, 
                                         transmission=sample_trans,
                                         direct_beam=direct_beam,
                                         direct_beam_transmission=direct_beam_trans, # note: background_trans
                                         masks=sample.masks,
                                         q_bins = q_bins,
                                         wavelength_bins = wavelength_bins,
                                         wavelength_bands = wavelength_bands)

CPU times: user 1.23 s, sys: 636 ms, total: 1.86 s
Wall time: 1.17 s


In [17]:
%%time
background_q_lambda = sans.to_q(data=background,
                                background = background, 
                                        transmission=background_trans,
                                        direct_beam=direct_beam,
                                        direct_beam_transmission=direct_beam_trans, # note: same as transmission
                                        masks=background.masks,
                                        q_bins = q_bins,
                                        wavelength_bins = wavelength_bins,
                                        wavelength_bands = wavelength_bands)

CPU times: user 1.29 s, sys: 628 ms, total: 1.92 s
Wall time: 891 ms


In [18]:
sample_q_reduce_wav = sc.sum(sample_q_lambda, 'wavelength')
background_q_reduce_wav = sc.sum(background_q_lambda, 'wavelength')

In [19]:
reduced_by_wavelength = sans.normalize_and_subtract(sample_q_reduce_wav, background_q_reduce_wav)

## Comoparison reduced vs reduced_by_wavelength

In [20]:
plot({'reduced': reduced, 'reduced_by_wavelength':reduced_by_wavelength})

In [21]:
sc.plot(sc.collapse(sample_q_lambda, keep='Q'))

In [22]:
sample_q_lambda['wavelength', 4:5]

<scipp.Dataset>
Dimensions: Sizes[wavelength:1, Q:103, ]
Coordinates:
  Q                         float64           [1/Å]  (Q [bin-edge])  [0.010000, 0.015000, ..., 0.520000, 0.525000]
  wavelength                float64             [Å]  (wavelength [bin-edge])  [6.000000, 7.000000]
Data:
  data                      float32         [counts]  (wavelength, Q)  [3408.049316, 5706.199707, ..., 0.000000, 0.000000]  [3408.049316, 5706.199707, ..., 0.000000, 0.000000]
    Attributes:
        Changer                 DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:2, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000]
Data:
                             string  [dimensionless]  (time)  ["IT", "JT"]

]
        Det_Temp_FLB            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [16.799999, 16.799999, 16.799999]

]
        Det_Temp_FLT            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.400000, 17.400000, 17.400000]

]
        Det_Temp_FRB            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.400000, 17.400000, 17.400000]

]
        Det_Temp_FRT            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.400000, 17.400000, 17.400000]

]
        Det_Temp_RLB            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.799999, 17.799999, 17.799999]

]
        Det_Temp_RLT            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.400000, 17.400000, 17.400000]

]
        Det_Temp_RRB            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.299999, 17.299999, 17.299999]

]
        Det_Temp_RRT            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.400000, 17.400000, 17.400000]

]
        Fast_Shutter            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: S

In [23]:
reduced_lambda = sans.normalize_and_subtract(sample_q_lambda, background_q_lambda)
sc.plot(sc.collapse(reduced_lambda, keep='Q'))

In [24]:
reduced_lambda['wavelength',4:5]

<scipp.DataArray>
Dimensions: Sizes[wavelength:1, Q:103, ]
Coordinates:
  Q                         float64           [1/Å]  (Q [bin-edge])  [0.010000, 0.015000, ..., 0.520000, 0.525000]
  wavelength                float64             [Å]  (wavelength [bin-edge])  [6.000000, 7.000000]
Data:
                            float64  [dimensionless]  (wavelength, Q)  [3.868223, 4.647683, ..., nan, nan]  [0.004710, 0.003944, ..., nan, nan]
Attributes:
  sample_position         vector_3_float64              [m]  ()  [(0.000000, 0.000000, 19.334000)]
  source_position         vector_3_float64              [m]  ()  [(0.000000, 0.000000, 0.000000)]

In [25]:
sample

<scipp.DataArray>
Dimensions: Sizes[spectrum:61440, tof:100, ]
Coordinates:
  base_position           vector_3_float64              [m]  (spectrum)  [(-0.519200, -0.480950, 23.281000), (-0.517162, -0.480950, 23.281000), ..., (0.517162, 0.482950, 23.281000), (0.519200, 0.482950, 23.281000)]
  position                vector_3_float64              [m]  (spectrum)  [(-0.611770, -0.399152, 23.281000), (-0.609732, -0.399152, 23.281000), ..., (0.424592, 0.564748, 23.281000), (0.426630, 0.564748, 23.281000)]
  sample_position         vector_3_float64              [m]  ()  [(0.000000, 0.000000, 19.334000)]
  source_position         vector_3_float64              [m]  ()  [(0.000000, 0.000000, 0.000000)]
  spectrum                    int32  [dimensionless]  (spectrum)  [9, 10, ..., 61447, 61448]
  tof                       float64            [µs]  (tof [bin-edge])  [0.000000, 1000.000000, ..., 99000.000000, 100000.000000]
Data:
                            float32         [counts]  (spectrum, tof)  [0.000000, 0.000000, ..., 0.000000, 0.000000]  [0.000000, 0.000000, ..., 0.000000, 0.000000]
Masks:
  mask_1_xml                   bool  [dimensionless]  (spectrum)  [True, True, ..., True, True]
  mask_2_xml                   bool  [dimensionless]  (spectrum)  [False, False, ..., False, False]
  mask_3_xml                   bool  [dimensionless]  (spectrum)  [False, False, ..., False, False]
  mask_4_xml                   bool  [dimensionless]  (spectrum)  [True, True, ..., False, False]
  mask_5_xml                   bool  [dimensionless]  (spectrum)  [False, False, ..., False, False]
  mask_6_xml                   bool  [dimensionless]  (spectrum)  [False, False, ..., False, False]
  mask_7_xml                   bool  [dimensionless]  (spectrum)  [False, False, ..., False, False]
  mask_8_xml                   bool  [dimensionless]  (spectrum)  [False, False, ..., False, False]
  mask_9_xml                   bool  [dimensionless]  (spectrum)  [False, False, ..., False, False]
Attributes:
  Changer                 DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:2, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000]
Data:
                             string  [dimensionless]  (time)  ["IT", "JT"]

]
  Det_Temp_FLB            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [16.799999, 16.799999, 16.799999]

]
  Det_Temp_FLT            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.400000, 17.400000, 17.400000]

]
  Det_Temp_FRB            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.400000, 17.400000, 17.400000]

]
  Det_Temp_FRT            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.400000, 17.400000, 17.400000]

]
  Det_Temp_RLB            DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    

## Comparison with Mantid

In [50]:
#scale factor at the end 0.02364 (from mantid)
mantid_q1d_file = '63114_rear_1D_2.0_10.0.txt'
mantid_q1d = load_rkh_q(filename=f'{path}/{mantid_q1d_file}')
reduced_scaled = sc.scalar(0.78) * reduced
plot({'scipp': reduced_scaled, 'mantid':mantid_q1d})

LoadRKH-[Notice] LoadRKH started
LoadRKH-[Notice] LoadRKH successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds


In [40]:
mantid_q1d_file_wav = '63114_rear_1D_2.0_3.0.txt'
mantid_q1d_wav_1 = load_rkh_q(filename=f'{path}/{mantid_q1d_file_wav}')
mantid_q1d_file_wav = '63114_rear_1D_3.0_4.0.txt'
mantid_q1d_wav_2 = load_rkh_q(filename=f'{path}/{mantid_q1d_file_wav}')
mantid_q1d_file_wav = '63114_rear_1D_4.0_5.0.txt'
mantid_q1d_wav_3 = load_rkh_q(filename=f'{path}/{mantid_q1d_file_wav}')
mantid_q1d_file_wav = '63114_rear_1D_5.0_6.0.txt'
mantid_q1d_wav_4 = load_rkh_q(filename=f'{path}/{mantid_q1d_file_wav}')
mantid_q1d_file_wav = '63114_rear_1D_6.0_7.0.txt'
mantid_q1d_wav_5 = load_rkh_q(filename=f'{path}/{mantid_q1d_file_wav}')
mantid_q1d_file_wav = '63114_rear_1D_7.0_8.0.txt'
mantid_q1d_wav_6 = load_rkh_q(filename=f'{path}/{mantid_q1d_file_wav}')
mantid_q1d_file_wav = '63114_rear_1D_8.0_9.0.txt'
mantid_q1d_wav_7 = load_rkh_q(filename=f'{path}/{mantid_q1d_file_wav}')
mantid_q1d_file_wav = '63114_rear_1D_9.0_10.0.txt'
mantid_q1d_wav_8 = load_rkh_q(filename=f'{path}/{mantid_q1d_file_wav}')


scipp_q1d_wav = sc.collapse(reduced_lambda, keep='Q')
scipp_q1d_wav = {f'scipp:{key}':val for key, val in scipp_q1d_wav.items()}
plot(scipp_q1d_wav)
#plot({'scipp': scipp_q1d_wav_1})

LoadRKH-[Notice] LoadRKH started
LoadRKH-[Notice] LoadRKH successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
LoadRKH-[Notice] LoadRKH started
LoadRKH-[Notice] LoadRKH successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
LoadRKH-[Notice] LoadRKH started
LoadRKH-[Notice] LoadRKH successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
LoadRKH-[Notice] LoadRKH started
LoadRKH-[Notice] LoadRKH successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace started
DeleteWorkspace-[Notice] DeleteWorkspace successful, Duration 0.00 seconds
LoadRKH-[Notice] LoadRKH started
LoadRKH-[Notice] LoadRKH successful, Duration 0.00 seconds
DeleteWorkspace-[Notice] DeleteWorkspace sta

In [41]:
scipp_q1d_wav['mantid_0'] = sc.scalar(1.55)*mantid_q1d_wav_1
plot({'mantid 2.0-3.0' : scipp_q1d_wav['mantid_0'], 'scipp 2.0-3.0': scipp_q1d_wav['scipp:wavelength:0']})


In [42]:
scipp_q1d_wav['mantid_1'] = sc.scalar(1.35)*mantid_q1d_wav_2
plot({'mantid 3.0-4.0' : scipp_q1d_wav['mantid_1'], 'scipp 3.0-4.0': scipp_q1d_wav['scipp:wavelength:1']})

In [43]:
scipp_q1d_wav['mantid_2'] = sc.scalar(1.35)*mantid_q1d_wav_3
plot({'mantid 4.0-5.0:' : scipp_q1d_wav['mantid_2'], 'scipp 4.0-5.0': scipp_q1d_wav['scipp:wavelength:2']})

In [44]:
scipp_q1d_wav['mantid_3'] = sc.scalar(1.25)*mantid_q1d_wav_4
plot({'mantid 5.0-6.0:' : scipp_q1d_wav['mantid_3'], 'scipp 5.0-6.0': scipp_q1d_wav['scipp:wavelength:3']})

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [64]:
scipp_q1d_wav['mantid_4'] = sc.scalar(1.2)*mantid_q1d_wav_5
plot({'mantid 6.0-7.0:' : scipp_q1d_wav['mantid_4'], 'scipp 6.0-7.0': scipp_q1d_wav['scipp:wavelength:4']})

In [55]:
scipp_q1d_wav['mantid_5'] = sc.scalar(1.2)*mantid_q1d_wav_6
plot({'mantid 7.0-8.0:' : scipp_q1d_wav['mantid_5'], 'scipp 7.0-8.0': scipp_q1d_wav['scipp:wavelength:5']})

In [59]:
scipp_q1d_wav['mantid_6'] = sc.scalar(0.9)*mantid_q1d_wav_7
plot({'mantid 8.0-9.0:' : scipp_q1d_wav['mantid_6'], 'scipp 8.0-9.0': scipp_q1d_wav['scipp:wavelength:6']})

In [63]:
scipp_q1d_wav['mantid_7'] = sc.scalar(0.55)*mantid_q1d_wav_8
plot({'mantid 9.0-10.0:' : scipp_q1d_wav['mantid_7'], 'scipp 9.0-10.0': scipp_q1d_wav['scipp:wavelength:7']})

In [49]:
help(sc.geometry.position)

Help on built-in function position in module scipp._scipp.core.geometry:

position(...) method of builtins.PyCapsule instance
    position(x: scipp._scipp.core.Variable, y: scipp._scipp.core.Variable, z: scipp._scipp.core.Variable) -> scipp._scipp.core.Variable
    
    Element-wise zip functionality to produce a vector_3_float64.
    
    :param x: Variable containing x component.
    :type x: Variable
    :param y: Variable containing y component.
    :type y: Variable
    :param z: Variable containing z component.
    :type z: Variable
    :raises: If the dtypes of inputs are not double precision floats.
    :return: Zip of input x, y and z. Output unit is same as input unit.
    :rtype: Variable

